# Prophet

In [ ]:
from prophet import Prophet

model = Prophet(weekly_seasonality=False, daily_seasonality=False, seasonality_mode='additive')

In [ ]:
ex_features = ['lag24', 'zap_lag24', 'zrodla_wiatrowe_lag24', "is_weekend", 'lag48', 'lag72', 'lag96', 'lag120', 'lag144','lag168', 'lag336', 'zap_lag48',
             'zrodla_wiatrowe_lag48', 'zap_lag72', 'zrodla_wiatrowe_lag72', 'zap_lag96', 'zrodla_wiatrowe_lag96',
              'zap_lag120', 'zrodla_wiatrowe_lag120', 'zap_lag144', 'zrodla_wiatrowe_lag144', 'zap_lag168', 'zrodla_wiatrowe_lag168',
              'zap_lag336', 'zrodla_wiatrowe_lag336', 'kod_dnia']

In [ ]:
for feature in ex_features:
    model.add_regressor(feature)
    
model.fit(df_train[["date", "value"] + ex_features].rename(columns={"date": "ds", "value": "y"}))

forecast = model.predict(df_test[["date", "value"] + ex_features].rename(columns={"date": "ds"}))

forecast_ = forecast.loc[:, forecast.columns.intersection(['ds','yhat'])]
forecast_ = forecast_.rename(columns={"ds": "date"})
df_test_ =  df_test.loc[:, df_test.columns.intersection(['date','value'])]
new_df = pd.merge(df_test_, forecast_, how='left', on=['date'])

new_df = new_df.set_index("date")

for c in new_df.columns:
    new_df[c] = new_df[c] * target_stdev + target_mean
    
df_test_plot = new_df

df_test_plot = df_test_plot.reset_index(drop=False)
df_test_plot = df_test_plot.rename(columns={"yhat": "Model forecast"})
